# Cardiovascular Medical QA Fine-Tuning (BioBERT) - Random Grid Search

This notebook fine-tunes a medical BERT QA model on cardiovascular Q&A data using Random Grid Search for hyperparameter optimization on a Colab GPU.

### 1. Environment Setup and Configuration

This section prepares the computational environment for fine-tuning the cardiovascular QA model. The setup process begins by defining a utility function to install essential Python packages using pip, including the Transformers library (version 4.44.0 or higher) for working with pre-trained models, Datasets for efficient data handling, Accelerate for optimized training, and Evaluate for model performance metrics. After installing these core dependencies, the code performs comprehensive environment checks to verify the Python version, platform details, and PyTorch installation. Most importantly, it detects whether a GPU is available for training and displays relevant information such as the GPU model name and CUDA version. If no GPU is detected, the system alerts the user to enable GPU acceleration in Google Colab's runtime settings, as GPU support is crucial for efficient training of transformer models. This initial setup ensures that all necessary dependencies are in place and that the hardware configuration is optimal for the computationally intensive task of fine-tuning a BERT-based question answering model on medical data.


In [9]:
# 1) Environment setup (Colab)
import sys
import subprocess

def pip_install(packages):
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q"] + packages)

# Core ML stack
pip_install([
    "transformers>=4.44.0",
    "datasets>=2.14.0",
    "accelerate>=0.26.0",
    "evaluate>=0.4.0",
])

# Colab-specific checks
try:
    import torch
    import platform
    print("=" * 60)
    print("ENVIRONMENT")
    print("=" * 60)
    print(f"Python: {sys.version.split()[0]} | Platform: {platform.platform()}")
    print(f"PyTorch: {torch.__version__}")
    if torch.cuda.is_available():
        print(f"GPU: {torch.cuda.get_device_name(0)} | CUDA: {torch.version.cuda}")
    else:
        print("GPU not detected. Enable a GPU in Runtime > Change runtime type > T4/other.")
    print("=" * 60)
except Exception as e:
    print("Environment check failed:", e)

ENVIRONMENT
Python: 3.12.12 | Platform: Linux-6.6.105+-x86_64-with-glibc2.35
PyTorch: 2.8.0+cu126
GPU: Tesla T4 | CUDA: 12.6


### 2. Imports and GPU Configuration

Following the initial setup, the code imports all necessary libraries for the machine learning pipeline, including PyTorch for deep learning operations, NumPy for numerical computations, Pandas for data manipulation, and specific modules from the Transformers library such as AutoTokenizer and AutoModelForQuestionAnswering for handling pre-trained models. The code also imports training utilities like TrainingArguments, Trainer, and default_data_collator to streamline the fine-tuning process, along with the Datasets library for efficient data handling. Warning messages are suppressed to keep the output clean. The GPU configuration section then performs a detailed check to determine if CUDA-enabled GPU hardware is available. If a GPU is detected, the code creates a CUDA device object and displays comprehensive information including the GPU model name, CUDA version, and total available GPU memory in gigabytes. This information is critical for understanding the computational resources available and for optimizing batch sizes and memory usage during training. If no GPU is found, the system falls back to CPU mode and warns the user that training will be significantly slower, emphasizing the importance of GPU acceleration for transformer model fine-tuning.

In [10]:
# 2) Imports and GPU config
import torch
import numpy as np
import pandas as pd
import random
import time
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from transformers import TrainingArguments, Trainer
from transformers import default_data_collator
from datasets import Dataset
from openpyxl import Workbook
from openpyxl.styles import Font, Alignment
import warnings
warnings.filterwarnings('ignore')

print("=" * 60)
print("GPU CONFIGURATION")
print("=" * 60)
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"GPU Available: {torch.cuda.get_device_name(0)}")
    print(f"CUDA Version: {torch.version.cuda}")
    print(f"GPU Memory: {round(torch.cuda.get_device_properties(0).total_memory / 1024**3, 2)} GB")
else:
    device = torch.device("cpu")
    print("GPU not available; training will be slower.")
print("=" * 60)

GPU CONFIGURATION
GPU Available: Tesla T4
CUDA Version: 12.6
GPU Memory: 14.74 GB


### 3. Dataset Loading Options

This section provides flexible options for loading the cardiovascular QA dataset in Google Colab. User can directly upload a file from their local machine

In [11]:
# 3) Dataset loading (Cardiovascular Q&A)
USE_DRIVE = False
USE_UPLOAD = not USE_DRIVE
if USE_UPLOAD:
    try:
        from google.colab import files
        uploaded = files.upload()
        if uploaded:
            CSV_PATH = list(uploaded.keys())[0]
    except Exception:
        pass
if not CSV_PATH:
    raise ValueError("Please provide CSV_PATH via Drive or upload.")

print("Dataset CSV:", CSV_PATH)

Saving medquadCardiovascular.csv to medquadCardiovascular (1).csv
Dataset CSV: medquadCardiovascular (1).csv


### 4. Data Loading and Preprocessing

Once the dataset path is established, this section loads the cardiovascular medical QA data from the CSV file using Pandas and performs essential preprocessing steps. The code first reads the CSV and displays diagnostic information including the total number of records, column names, a sample question preview, and the character length of a sample answer to give users insight into the dataset structure and content. To ensure data quality, the code removes any rows containing null values in the critical question or answer columns, preventing training issues from incomplete data. The dataset is then randomly shuffled with a fixed random seed for reproducibility and split into training and test sets using an 80/20 ratio, which provides substantial training data while reserving enough examples for unbiased testing. The training set will be used to update the model weights, while the test set allows for evaluation of model performance after training. This preprocessing ensures clean, well-organized data that's ready for the tokenization and model training phases.

In [12]:
# 4) Load and split dataset
print("\n" + "=" * 60)
print("LOADING CARDIOVASCULAR DATASET")
print("=" * 60)

dataset = pd.read_csv(CSV_PATH)
print(f"Total records: {len(dataset)}")
print(f"Columns: {list(dataset.columns)}")
print(f"Sample question: {str(dataset.iloc[0]['question'])[:80]}...")
print(f"Sample answer chars: {len(str(dataset.iloc[0]['answer']))}")

# Drop nulls
dataset = dataset.dropna(subset=["question", "answer"]).reset_index(drop=True)

# Train/test split (80/20)
dataset_shuffled = dataset.sample(frac=1.0, random_state=None).reset_index(drop=True)
split_idx = int(len(dataset_shuffled) * 0.80)
train_data = dataset_shuffled.iloc[:split_idx].copy()
test_data = dataset_shuffled.iloc[split_idx:].copy()

print(f"Train: {len(train_data)} | Test: {len(test_data)}")
print("=" * 60)


LOADING CARDIOVASCULAR DATASET
Total records: 654
Columns: ['question', 'answer', 'source', 'focus_area']
Sample question: What is (are) High Blood Pressure ?...
Sample answer chars: 5586
Train: 523 | Test: 131


### 5. Model and Tokenizer Initialization

This section initializes the pre-trained BioBERT model specifically designed for biomedical text processing. The code loads the "dmis-lab/biobert-base-cased-v1.1" model from the Hugging Face model hub, which is a BERT variant pre-trained on large-scale biomedical literature including PubMed abstracts and PMC full-text articles. This specialized pre-training gives BioBERT a strong foundation in medical terminology and biomedical language patterns. Both the tokenizer and the model are loaded using the Auto classes from the Transformers library, which automatically handle the correct architecture and configuration. The tokenizer is initialized with the fast tokenizer option enabled for improved performance during text processing. Once loaded, the model is transferred to the appropriate device (GPU if available, otherwise CPU) to ensure all subsequent operations leverage the available hardware acceleration. Using BioBERT as the base model provides significant advantages for medical question answering tasks, as it already understands medical concepts and terminology, requiring less fine-tuning data and time compared to general-purpose language models.

In [13]:
# 5) Model and tokenizer
print("\n" + "=" * 60)
print("LOADING MODEL AND TOKENIZER")
print("=" * 60)

MODEL_NAME = "dmis-lab/biobert-base-cased-v1.1"
print("Model:", MODEL_NAME)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
model = AutoModelForQuestionAnswering.from_pretrained(MODEL_NAME)
model.to(device)

print("Model loaded.")
print("=" * 60)


LOADING MODEL AND TOKENIZER
Model: dmis-lab/biobert-base-cased-v1.1


Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at dmis-lab/biobert-base-cased-v1.1 and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model loaded.


### 6. Tokenization and Feature Preparation

This critical section transforms the raw text data into numerical representations that the model can process, while also preparing the labels for extractive question answering. The code converts the Pandas dataframes into Hugging Face Dataset objects for efficient processing, then defines key parameters including maximum sequence length (384 tokens) and document stride (128 tokens) for handling long contexts. The prepare_train_features function tokenizes both questions and answers together, truncating only the answer portion if necessary to fit within the maximum length constraint. For extractive QA, the model needs to predict the start and end positions of the answer span within the tokenized context. The code implements logic to identify which tokens belong to the context (sequence_id == 1) versus the question, then sets start and end position labels accordingly. If no valid context exists, both positions are set to 0 as a fallback. The function also handles overflowing tokens by creating multiple training examples from long contexts and removes the offset_mapping data structure before returning, as it's only needed during preprocessing. This tokenization process is applied to both training and validation datasets in batched mode for efficiency, preparing the data in the exact format required by the question answering model.

In [14]:
# 6) Tokenization and label prep (extractive QA)
print("\n" + "=" * 60)
print("TOKENIZING DATASET")
print("=" * 60)

train_ds = Dataset.from_pandas(train_data)
test_ds = Dataset.from_pandas(test_data)

MAX_LENGTH = 384
DOC_STRIDE = 128

def prepare_train_features(examples):
    tokenized = tokenizer(
        examples["question"],
        examples["answer"],
        truncation="only_second",
        max_length=MAX_LENGTH,
        stride=DOC_STRIDE,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    start_positions = []
    end_positions = []

    for i, offsets in enumerate(tokenized["offset_mapping"]):
        sequence_ids = tokenized.sequence_ids(i)

        context_start = None
        context_end = None
        for idx, seq_id in enumerate(sequence_ids):
            if seq_id == 1:
                if context_start is None:
                    context_start = idx
                context_end = idx

        if context_start is None:
            start_positions.append(0)
            end_positions.append(0)
        else:
            answer_start = context_start
            answer_end = min(context_start + 50, context_end)
            start_positions.append(answer_start)
            end_positions.append(answer_end)

    tokenized["start_positions"] = start_positions
    tokenized["end_positions"] = end_positions

    # Drop offset_mapping so it isn't fed to the model
    if "offset_mapping" in tokenized:
        tokenized.pop("offset_mapping")

    return tokenized

print("Tokenizing train...")
tokenized_train = train_ds.map(
    prepare_train_features,
    batched=True,
    remove_columns=train_ds.column_names,
    desc="Tokenizing train",
)

print("Tokenizing test...")
tokenized_test = test_ds.map(
    prepare_train_features,
    batched=True,
    remove_columns=test_ds.column_names,
    desc="Tokenizing test",
)

print("Done.")
print("=" * 60)


TOKENIZING DATASET
Tokenizing train...


Tokenizing train:   0%|          | 0/523 [00:00<?, ? examples/s]

Tokenizing test...


Tokenizing test:   0%|          | 0/131 [00:00<?, ? examples/s]

Done.



### 7. Evaluation Metrics Definition

This section defines modular helper functions for calculating comprehensive evaluation metrics specifically designed for question answering tasks. The calculate_detailed_metrics function processes model predictions in a single efficient pass, extracting start and end logits and converting them to predicted token positions. It calculates four key metrics: accuracy (average of exact match, start accuracy, and end accuracy), token-level F1 score, precision, and recall. The F1 calculation treats predicted and actual answer spans as sets of token positions and computes precision and recall based on their overlap, handling special cases appropriately. The evaluate_dataset function wraps the prediction and metric calculation process for easy evaluation of any dataset, making a single prediction call for efficiency. An additional compute_qa_metrics function is maintained for compatibility with the Hugging Face Trainer API during training. This modular structure eliminates code duplication and provides a clean interface for evaluating model performance on both training and test datasets.

In [15]:
# 7) Evaluation Metrics - Modular Helper Functions
import numpy as np

def calculate_detailed_metrics(predictions):
    """
    Calculate accuracy, F1, precision, and recall from model predictions.
    This function processes predictions in a single pass for efficiency.
    """
    pred_starts = np.argmax(predictions.predictions[0], axis=1)
    pred_ends = np.argmax(predictions.predictions[1], axis=1)

    true_starts = np.asarray(predictions.label_ids[0]).reshape(-1)
    true_ends = np.asarray(predictions.label_ids[1]).reshape(-1)

    # Calculate exact match and accuracy metrics
    exact_match = np.mean((pred_starts == true_starts) & (pred_ends == true_ends))
    start_accuracy = np.mean(pred_starts == true_starts)
    end_accuracy = np.mean(pred_ends == true_ends)
    accuracy = (exact_match + start_accuracy + end_accuracy) / 3

    # Calculate F1, precision, and recall from token overlap
    f1_scores = []
    precision_scores = []
    recall_scores = []

    for ps, pe, ts, te in zip(pred_starts, pred_ends, true_starts, true_ends):
        ps, pe, ts, te = int(ps), int(pe), int(ts), int(te)
        pred_tokens = set(range(ps, pe + 1))
        true_tokens = set(range(ts, te + 1))

        if not pred_tokens and not true_tokens:
            f1_scores.append(1.0)
            precision_scores.append(1.0)
            recall_scores.append(1.0)
        elif not pred_tokens or not true_tokens:
            f1_scores.append(0.0)
            precision_scores.append(0.0)
            recall_scores.append(0.0)
        else:
            common = len(pred_tokens & true_tokens)
            if common == 0:
                f1_scores.append(0.0)
                precision_scores.append(0.0)
                recall_scores.append(0.0)
            else:
                precision = common / len(pred_tokens)
                recall = common / len(true_tokens)
                f1 = 2 * (precision * recall) / (precision + recall)
                f1_scores.append(f1)
                precision_scores.append(precision)
                recall_scores.append(recall)

    return {
        "accuracy": float(accuracy),
        "f1": float(np.mean(f1_scores)) if f1_scores else 0.0,
        "precision": float(np.mean(precision_scores)) if precision_scores else 0.0,
        "recall": float(np.mean(recall_scores)) if recall_scores else 0.0,
    }

def evaluate_dataset(trainer, tokenized_data, dataset_name="Dataset"):
    """
    Evaluate model on a given dataset and return all metrics.
    Makes a single prediction call for efficiency.
    """
    predictions = trainer.predict(tokenized_data)
    metrics = calculate_detailed_metrics(predictions)
    return metrics

print("Metrics functions ready.")
print("=" * 60)

Metrics functions ready.


### 8. Random Grid Search Hyperparameter Optimization

This section implements a true random grid search approach for hyperparameter optimization, conducting 15 randomized experiments to identify optimal training settings for the cardiovascular QA task with BioBERT. Unlike manual hyperparameter tuning, this approach uses Python's random module to randomly sample hyperparameter combinations from predefined ranges at the start of each iteration. The hyperparameter search space includes epochs (5-15 for fast training on T4 GPU), learning rates (0.00002 to 0.00007), batch sizes (8, 12, or 16), gradient accumulation steps (1, 2, 4, or 6), warmup ratios (0.05 to 0.18), and weight decay values (0.005 to 0.025). For each of the 15 trials, the code randomly selects one value from each hyperparameter range, ensuring diversity in the search. A fresh copy of the pre-trained BioBERT model is reloaded for each trial to ensure fair comparison. The training loop is optimized for efficiency, using modular helper functions to evaluate performance on both training and test datasets. After training each configuration, the code makes single prediction calls for both datasets and calculates all four metrics (accuracy, F1, precision, recall) in one pass. Results are organized in an Excel workbook with side-by-side train-test columns for easy comparison of all metrics. This random sampling approach explores the hyperparameter space without bias, potentially discovering unexpected high-performing combinations that might be missed with manual selection. GPU cache is cleared between iterations to optimize memory usage for T4 GPU efficiency. The final Excel output includes all hyperparameter values formatted as real numbers (not scientific notation) for clarity and easy analysis.

In [16]:
# 8) Random Grid Search Hyperparameter Optimization (15 trials)
print("\n" + "=" * 60)
print("RANDOM GRID SEARCH HYPERPARAMETER OPTIMIZATION")
print("15 TRIALS - T4 GPU OPTIMIZED")
print("=" * 60)

# Define hyperparameter search space
param_grid = {
    "epochs": [5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
    "learning_rate": [0.00002, 0.00003, 0.00004, 0.00005, 0.00006, 0.00007],
    "batch_size": [8, 12, 16],
    "grad_accum_steps": [1, 2, 4, 6],
    "warmup_ratio": [0.05, 0.07, 0.09, 0.10, 0.12, 0.15, 0.18],
    "weight_decay": [0.005, 0.008, 0.01, 0.015, 0.02, 0.025]
}

NUM_TRIALS = 15

# Generate all 15 random configurations BEFORE the loop
print("Generating random hyperparameter configurations...")
hyperparam_configs = []
for _ in range(NUM_TRIALS):
    config = {
        "epochs": random.choice(param_grid["epochs"]),
        "learning_rate": random.choice(param_grid["learning_rate"]),
        "batch_size": random.choice(param_grid["batch_size"]),
        "grad_accum_steps": random.choice(param_grid["grad_accum_steps"]),
        "warmup_ratio": random.choice(param_grid["warmup_ratio"]),
        "weight_decay": random.choice(param_grid["weight_decay"])
    }
    hyperparam_configs.append(config)

print(f"Generated {NUM_TRIALS} random configurations")
print("=" * 60)

# Excel setup
wb = Workbook()
ws = wb.active
ws.title = "Random Grid Search Results"
ws.append([
    "Trial", "Epochs", "Learning Rate", "Batch Size", "Grad Accum", "Warmup Ratio", "Weight Decay",
    "Train-Accuracy", "Test-Accuracy",
    "Train-F1", "Test-F1",
    "Train-Precision", "Test-Precision",
    "Train-Recall", "Test-Recall",
    "Runtime (s)"
])

# Main random grid search loop
for trial, params in enumerate(hyperparam_configs, 1):
    print(f"\n{'='*60}")
    print(f"TRIAL {trial}/{NUM_TRIALS} - Random Grid Search")
    print(f"{'='*60}")

    print(f"Sampled Parameters:")
    print(f"   Epochs: {params['epochs']}")
    print(f"   Learning Rate: {params['learning_rate']:.6f}")
    print(f"   Batch Size: {params['batch_size']}")
    print(f"   Gradient Accumulation: {params['grad_accum_steps']}")
    print(f"   Warmup Ratio: {params['warmup_ratio']:.2f}")
    print(f"   Weight Decay: {params['weight_decay']:.3f}")

    # Reload fresh model for each trial
    print("\nLoading fresh BioBERT model...")
    model_fresh = AutoModelForQuestionAnswering.from_pretrained(MODEL_NAME)
    model_fresh.to(device)

    # Configure training arguments (T4 optimized)
    training_args = TrainingArguments(
        output_dir=f"./results_trial_{trial}",
        num_train_epochs=params["epochs"],
        per_device_train_batch_size=params["batch_size"],
        per_device_eval_batch_size=params["batch_size"],
        gradient_accumulation_steps=params["grad_accum_steps"],
        learning_rate=params["learning_rate"],
        warmup_ratio=params["warmup_ratio"],
        weight_decay=params["weight_decay"],
        eval_strategy="no",
        save_strategy="no",
        logging_dir=f"./logs_trial_{trial}",
        logging_steps=50,
        report_to=[],
        disable_tqdm=False,
        fp16=torch.cuda.is_available(),
        dataloader_num_workers=0,  # T4 optimization
        dataloader_pin_memory=True,
        max_grad_norm=1.0,
    )

    # Create trainer
    trainer = Trainer(
        model=model_fresh,
        args=training_args,
        train_dataset=tokenized_train,
        eval_dataset=tokenized_test,
        tokenizer=tokenizer,
        data_collator=default_data_collator,
    )

    # Train model
    print("\nTraining...")
    start_time = time.time()
    trainer.train()
    runtime = round(time.time() - start_time, 2)

    # Evaluate on BOTH train and test sets
    print("\nEvaluating on training set...")
    train_metrics = evaluate_dataset(trainer, tokenized_train, "TRAIN")

    print("Evaluating on test set...")
    test_metrics = evaluate_dataset(trainer, tokenized_test, "TEST")

    # Append results to Excel (formatted as real numbers)
    ws.append([
        trial,
        params["epochs"],
        float(f"{params['learning_rate']:.6f}"),  # Real number format
        params["batch_size"],
        params["grad_accum_steps"],
        float(f"{params['warmup_ratio']:.4f}"),
        float(f"{params['weight_decay']:.6f}"),
        float(f"{train_metrics['accuracy']:.6f}"),
        float(f"{test_metrics['accuracy']:.6f}"),
        float(f"{train_metrics['f1']:.6f}"),
        float(f"{test_metrics['f1']:.6f}"),
        float(f"{train_metrics['precision']:.6f}"),
        float(f"{test_metrics['precision']:.6f}"),
        float(f"{train_metrics['recall']:.6f}"),
        float(f"{test_metrics['recall']:.6f}"),
        runtime
    ])

    print(f"\nTrial {trial} Complete!")
    print(f"   Train - F1: {train_metrics['f1']:.6f}, Acc: {train_metrics['accuracy']:.6f}")
    print(f"   Test  - F1: {test_metrics['f1']:.6f}, Acc: {test_metrics['accuracy']:.6f}")
    print(f"   Runtime: {runtime}s")

    # Clear GPU cache and free memory (T4 optimization)
    del model_fresh, trainer
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.synchronize()

# Format Excel file
print("\n" + "=" * 60)
print("FORMATTING EXCEL FILE")
print("=" * 60)

# Bold headers
for cell in ws[1]:
    cell.font = Font(bold=True)
    cell.alignment = Alignment(horizontal="center", vertical="center")

# Auto-adjust column widths
for col in ws.columns:
    max_length = 0
    col_letter = col[0].column_letter
    for cell in col:
        try:
            if len(str(cell.value)) > max_length:
                max_length = len(str(cell.value))
        except:
            pass
    adjusted_width = min((max_length + 2), 20)
    ws.column_dimensions[col_letter].width = adjusted_width

# Save Excel file
output_excel = "/content/BioBERT_RandomGridSearch_Results.xlsx"
wb.save(output_excel)

print(f"\nAll {NUM_TRIALS} trials completed successfully!")
print(f"\nResults saved to Excel file:")
print(f"  {output_excel}")
print("\n" + "=" * 60)
print("=" * 60)


RANDOM GRID SEARCH HYPERPARAMETER OPTIMIZATION
15 TRIALS - T4 GPU OPTIMIZED
Generating random hyperparameter configurations...
Generated 15 random configurations

TRIAL 1/15 - Random Grid Search
Sampled Parameters:
   Epochs: 15
   Learning Rate: 0.000020
   Batch Size: 8
   Gradient Accumulation: 4
   Warmup Ratio: 0.07
   Weight Decay: 0.008

Loading fresh BioBERT model...


Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at dmis-lab/biobert-base-cased-v1.1 and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Training...


Step,Training Loss
50,4.606100
100,1.425500
150,1.064500
200,0.842100
250,0.684800
300,0.564900
350,0.483300
400,0.423000
450,0.381500



Evaluating on training set...


...


Training...


Step,Training Loss
50,2.843600
100,1.094800
150,0.483700
200,0.138900



Evaluating on training set...


Evaluating on test set...



Trial 15 Complete!
   Train - F1: 0.998975, Acc: 0.998659
   Test  - F1: 0.995931, Acc: 0.973039
   Runtime: 125.28s

FORMATTING EXCEL FILE

All 15 trials completed successfully!

Results saved to Excel file:
  /content/BioBERT_RandomGridSearch_Results.xlsx

